In [1]:
import torch
import torch.nn as nn
import pandas as pd

from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Using VAE for self-supervised task

The main idea of this approach is training unsupervised VAE by generating data similar to original. Then the output of VAE's encoder in latent space will go through a classification layer.

### Prepare the dataset

In [3]:
train_dataset = pd.read_csv('data/train_dataset.csv')
test_dataset = pd.read_csv('data/test_dataset.csv')

data_columns = ["V"+str(i) for i in range(1,29)]+["Amount"]
label_column ="Class"

X_train = train_dataset[data_columns]
X_test  = test_dataset[data_columns]

y_train = train_dataset[label_column]
y_test  = test_dataset[label_column]

In [5]:
x_train_tensor = torch.from_numpy(X_train.values).to(device)
y_train_tensor = torch.from_numpy(y_train.values).to(device)

x_test_tensor = torch.from_numpy(X_test.values).to(device)
y_test_tensor = torch.from_numpy(y_test.values).to(device)

Train_tensor = TensorDataset(x_train_tensor, y_train_tensor)
Test_tensor = TensorDataset(x_test_tensor, y_test_tensor)

Train_dataset = DataLoader(Train_tensor, batch_size=512, shuffle=True)
Test_dataset = DataLoader(Test_tensor, batch_size=512, shuffle=True)

## The Models

### VAE
This is the simplest version of VAE. You can add more layers, CNNs if you like.

In [50]:
class VAE(nn.Module):
    def __init__(self,input_dim=29, hidden_dim=20, latent_dim=6):
        super(VAE, self).__init__()
        
        # Encoder
        self.ln1_encoder = nn.Linear(input_dim, hidden_dim)
        
        self.mu_encoder = nn.Linear(hidden_dim, latent_dim)
        self.logvar_encoder = nn.Linear(hidden_dim, latent_dim)
        
        # Decoder
        self.ln1_decoder = nn.Linear(latent_dim, hidden_dim)
        self.ln2_decoder = nn.Linear(hidden_dim, input_dim)
        
        self.activation = nn.ReLU()
        
    def encode(self, x):
        h1 = self.activation(self.ln1_encoder(x))
        return self.mu_encoder(h1), self.logvar_encoder(h1)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decoder(self, z):
        h1 = self.activation(self.ln1_decoder(z))
        return torch.sigmoid(self.ln2_decoder(h1))
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

### ClassificationModel

In [51]:
class Classification(nn.Module):
    def __init__(self, input_dim=6):
        super(Classification, self).__init__()
        self.ln1 = nn.Linear(input_dim, input_dim)
        self.output = nn.Linear(input_dim, 1)
        self.activation = nn.ReLU()
    def forward(self, x):
        x = self.activation(self.ln1(x))
        return self.output(x)
                            

### The combination of 2 models

In [62]:
class VAE_Classification(nn.Module):
    def __init__(self, vae, classifier):
        super(VAE_Classification, self).__init__()
        self.vae = vae
        self.classifier = classifier
    
    def forward(self, x,):
    
        mu, logvar = self.vae.encode(x)
        return self.classifier(self.vae.reparameterize(mu,logvar))

### Loss funtion for VAE
$$
L_{loss} = L_{recon} + L_{D_{KL}}
$$

In [53]:
def VAEloss(x, x_hat, mu, logvar):
    BCE = nn.MSELoss()(x_hat, x)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(),dim=1)
    return BCE + KLD.mean()

## Parameter

In [56]:
epochs = 100

#opitmizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

## Training

### Training VAE

In [58]:
vae = VAE().to(device)
opitmizer = torch.optim.Adam(vae.parameters(), lr=0.001)

In [60]:
for epoch in range(epochs):
    losses = 0
    for x, y in tqdm(Train_dataset):
        opitmizer.zero_grad()
        x_hat, mu, log_var = vae(x.float())
        loss = VAEloss(x.float(), x_hat, mu, log_var)
        # loss = criterion(output, y.float().view(-1, 1))
        # loss += kl
        loss.backward()
        opitmizer.step()
        losses += loss.item()
    print(f"Epoch {epoch} Loss: {losses/len(Train_dataset)}")
        

100%|██████████| 889/889 [00:05<00:00, 148.84it/s]


Epoch 0 Loss: 12572.512447052517


100%|██████████| 889/889 [00:05<00:00, 152.21it/s]


Epoch 1 Loss: 11067.14281485078


100%|██████████| 889/889 [00:05<00:00, 150.49it/s]


...

Epoch 97 Loss: 10374.760232485587


100%|██████████| 889/889 [00:05<00:00, 148.44it/s]


Epoch 98 Loss: 10264.955586728416


100%|██████████| 889/889 [00:06<00:00, 144.86it/s]

Epoch 99 Loss: 10264.386522668554


### Training model

In [63]:
net = VAE_Classification(vae, Classification()).to(device)
opitmizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [65]:
for epoch in range(epochs):
    for x, y in tqdm(Train_dataset):
        opitmizer.zero_grad()
        output = net(x.float())

        loss = criterion(output, y.float().view(-1, 1))
        loss.backward()
        opitmizer.step()

    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in tqdm(Test_dataset):
            output = net(x.float())
            predicted = (output >= 0.5).float()
            total += y.size(0)
            correct += (predicted == y.float().view(-1, 1)).sum().item()
        print(f"Epoch {epoch} Accuracy: {correct*100/total:.4f}%")

100%|██████████| 223/223 [00:00<00:00, 253.94it/s]


Epoch 0 Accuracy: 96.7633%


100%|██████████| 223/223 [00:01<00:00, 181.03it/s]


Epoch 1 Accuracy: 97.9240%


100%|██████████| 223/223 [00:01<00:00, 220.27it/s]


...

Epoch 30 Accuracy: 99.8136%


100%|██████████| 223/223 [00:00<00:00, 247.03it/s]


Epoch 31 Accuracy: 99.8259%


 27%|██▋       | 240/889 [00:01<00:04, 141.47it/s]


KeyboardInterrupt: 

In [ ]:
# 99.83 ok?